# Topics

- 3D
- Falling bricks

In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
import json
import time

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("22-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'2,6,9~5,6,9\n9,3,147~9,3,150\n1,0,190~4,0,190\n4,8,37~6,8,37\n0,4,7~0,5,7\n6,2,184~6,4,184\n7,0,157~8,0,157\n4,5,216~6,5,216\n5,0,359~5,1,359\n8,2,64~8,5,64\n1,6,42~2,6,42\n3,4,48~5,4,48\n7,9,219~7,9,219\n0,2,224~0,5,224\n3,3,240~7,3,240\n4,6,232~4,8,232\n2,8,3~2,9,3\n6,2,91~6,3,91\n7,2,210~7,4,210\n3,4,195~5,4,195\n1,3,239~3,3,239\n8,3,152~8,5,152\n6,3,135~8,3,135\n1,1,263~1,2,263\n0,0,277~0,1,277\n3,0,262~3,1,262\n8,1,82~8,4,82\n0,6,244~2,6,244\n2,0,103~2,4,103\n6,1,82~6,3,82\n8,2,257~8,5,257\n4,6,136~6,6,136\n7,1,333~7,2,333\n5,8,137~5,9,137\n3,0,144~3,3,144\n3,6,12~3,7,12\n2,3,362~5,3,362\n8,0,353~8,3,353\n9,2,191~9,4,191\n9,4,102~9,7,102\n3,5,170~3,7,170\n4,8,316~5,8,316\n8,7,41~8,9,41\n1,9,331~3,9,331\n5,2,110~5,2,111\n4,2,229~6,2,229\n3,5,73~3,5,74\n1,9,128~1,9,130\n6,3,234~7,3,234\n0,1,238~0,3,238\n8,2,5~8,3,5\n4,5,308~4,7,308\n9,3,203~9,5,203\n1,4,228~1,7,228\n3,9,359~5,9,359\n5,2,200~5,3,200\n6,2,297~8,2,297\n4,2,267~6,2,267\n3,7,28~5,7,28\n9,5,73~9,8,73\n0,7,237~1,7,237\n3,

In [4]:
test_data_raw = r"""1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""




Step = namedtuple("Step", ["direction", "number", "color"])
Point = namedtuple("Point", ["x", "y", "z"])

def cmp_items(a,b):
    
    if a[0].z < b[0].z:
        return -1
    elif a[0].z > b[0].z:
        return 1
    else:
        return 0

def preprocess_data (data):
    # dtype='U10'

    rows = [ (row.split("~")[0], row.split("~")[1]) for row in data.split("\n")]
    rows = [  (Point(*[int(i) for i in a.split(",")]), Point(*[int(i) for i in b.split(",")]))   for a, b in rows]
    rows.sort(key=cmp_to_key(cmp_items))
    return rows

test_data = preprocess_data(test_data_raw)
display(test_data)

[(Point(x=1, y=0, z=1), Point(x=1, y=2, z=1)),
 (Point(x=0, y=0, z=2), Point(x=2, y=0, z=2)),
 (Point(x=0, y=2, z=3), Point(x=2, y=2, z=3)),
 (Point(x=0, y=0, z=4), Point(x=0, y=2, z=4)),
 (Point(x=2, y=0, z=5), Point(x=2, y=2, z=5)),
 (Point(x=0, y=1, z=6), Point(x=2, y=1, z=6)),
 (Point(x=1, y=1, z=8), Point(x=1, y=1, z=9))]

In [5]:
data = preprocess_data(data_raw)
data

[(Point(x=1, y=9, z=1), Point(x=3, y=9, z=1)),
 (Point(x=2, y=1, z=1), Point(x=2, y=3, z=1)),
 (Point(x=1, y=1, z=2), Point(x=1, y=3, z=2)),
 (Point(x=5, y=1, z=2), Point(x=7, y=1, z=2)),
 (Point(x=3, y=7, z=2), Point(x=5, y=7, z=2)),
 (Point(x=3, y=6, z=2), Point(x=5, y=6, z=2)),
 (Point(x=8, y=2, z=2), Point(x=8, y=4, z=2)),
 (Point(x=0, y=4, z=2), Point(x=0, y=6, z=2)),
 (Point(x=6, y=2, z=2), Point(x=6, y=2, z=4)),
 (Point(x=0, y=0, z=2), Point(x=2, y=0, z=2)),
 (Point(x=4, y=5, z=2), Point(x=8, y=5, z=2)),
 (Point(x=2, y=8, z=3), Point(x=2, y=9, z=3)),
 (Point(x=3, y=2, z=3), Point(x=4, y=2, z=3)),
 (Point(x=0, y=0, z=3), Point(x=0, y=2, z=3)),
 (Point(x=5, y=6, z=3), Point(x=5, y=9, z=3)),
 (Point(x=4, y=0, z=3), Point(x=7, y=0, z=3)),
 (Point(x=6, y=8, z=3), Point(x=8, y=8, z=3)),
 (Point(x=1, y=4, z=3), Point(x=1, y=6, z=3)),
 (Point(x=7, y=1, z=3), Point(x=7, y=3, z=3)),
 (Point(x=0, y=9, z=3), Point(x=0, y=9, z=6)),
 (Point(x=4, y=8, z=4), Point(x=5, y=8, z=4)),
 (Point(x=2, 

In [97]:
def solution (data, verbose=False):
    
    def calc_orientation (brick):
        if brick[0].x != brick[1].x:
            return "x"
        elif brick[0].y != brick[1].y:
            return "y"
        elif brick[0].z != brick[1].z:
            return "z"
        else:
            return "0"

    dim_max = [0,0,0]
    points = [point for brick in data for point in brick]
    for i in range(3):
        dim_max[i] = max([point[i] for point in points])+1


    structure = np.zeros(dim_max, dtype=str)
    stack = np.zeros(dim_max[:2], dtype=int)

    for idx, brick in enumerate(data):
        if verbose:
            print(brick)
        orientation = calc_orientation(brick)
        if orientation in ("x", "0"):
            y = brick[0].y
            max_z = max(stack[brick[0].x:brick[1].x+1, y])
            stack[brick[0].x:brick[1].x+1, y] =  max_z + 1 
            
        elif orientation == "y":
            x = brick[0].x
            max_z = max(stack[x, brick[0].y:brick[1].y+1])
            stack[x, brick[0].y:brick[1].y+1] =  max_z + 1 

        elif orientation == "z":
            x = brick[0].x
            y = brick[0].y
            max_z = stack[x,y]
            stack[x,y] = max_z + (brick[1].z - brick[0].z + 1)
        
        structure[brick[0].x : brick[1].x+1, brick[0].y : brick[1].y+1, max_z+1 : max_z+1 + (brick[1].z - brick[0].z + 1)  ] = chr(ord("A") +idx)
        # print(structure[0,:,:])
        # print(structure[:,0,:])

    support_map = defaultdict(list)
    supporting_map = defaultdict(list)


    for idx, brick in enumerate(data):
        label = chr(ord("A") +idx)
        locations = np.where(structure == label)
        
        supporting = np.unique(structure[locations[0], locations[1], locations[2]+1]).tolist()
        if "" in supporting:
            supporting.remove("")
        if label in supporting:
            supporting.remove(label)

        supporting_map[label] = supporting

        for sup in supporting:
            if not sup in ("", label):
                support_map[sup] = support_map[sup] + [label]
        if verbose:
            print(label, "supports", supporting)
    
    if verbose:
        display(support_map)

    to_disintigrate = []
    for idx, brick in enumerate(data):
        label = chr(ord("A") +idx)
        disintigrate = True
        for supporting in supporting_map[label]:
            if len(support_map[supporting]) < 2:
                disintigrate = False

        if disintigrate:
            to_disintigrate += [label]
    
    return to_disintigrate
    

sol  = solution(data, verbose=False)
# sol = solution(test_data, verbose=False)

display(sol)
display(len(sol))
# display(sum(sol))


['C',
 'E',
 'F',
 'K',
 'L',
 'N',
 'P',
 'R',
 'T',
 'V',
 'X',
 'Z',
 '\\',
 'g',
 'i',
 'j',
 'k',
 'n',
 'o',
 'p',
 's',
 't',
 'u',
 'w',
 'x',
 '|',
 '}',
 '\x85',
 '\x87',
 '\x8a',
 '\x8b',
 '\x97',
 '\x98',
 '\x9a',
 '\x9c',
 '\x9d',
 '¡',
 '¢',
 '¨',
 '¬',
 '³',
 '´',
 '¶',
 '¹',
 'º',
 '¿',
 'À',
 'Á',
 'Ã',
 'Æ',
 'É',
 'Ë',
 'Ì',
 'Î',
 'Õ',
 'Þ',
 'ã',
 'å',
 'è',
 'ë',
 'ñ',
 'õ',
 '÷',
 'ø',
 'ù',
 'ú',
 'ü',
 'Ā',
 'Ă',
 'Ĉ',
 'Ċ',
 'č',
 'Đ',
 'đ',
 'Ě',
 'Ğ',
 'Ģ',
 'ĥ',
 'Ĩ',
 'ī',
 'Ĭ',
 'ĭ',
 'Į',
 'ĳ',
 'ĺ',
 'ŀ',
 'ł',
 'ņ',
 'Ň',
 'Ŋ',
 'ŋ',
 'Ŏ',
 'Ŗ',
 'ŗ',
 'Ř',
 'ř',
 'Ś',
 'Ŝ',
 'š',
 'ũ',
 'Ŭ',
 'ŭ',
 'ů',
 'Ű',
 'Ŷ',
 'ŷ',
 'Ÿ',
 'Ż',
 'ż',
 'ſ',
 'Ƃ',
 'ƌ',
 'Ɩ',
 'ƙ',
 'Ɯ',
 'ƞ',
 'Ɵ',
 'ƣ',
 'Ʀ',
 'Ƨ',
 'Ʃ',
 'ƭ',
 'Ʋ',
 'Ʒ',
 'Ƹ',
 'ƹ',
 'Ƽ',
 'ǁ',
 'ǂ',
 'ǆ',
 'Ǌ',
 'ǋ',
 'ǎ',
 'Ǖ',
 'Ǘ',
 'Ǚ',
 'ǚ',
 'ǝ',
 'Ǣ',
 'ǣ',
 'Ǧ',
 'ǰ',
 'ǵ',
 'Ǻ',
 'ǻ',
 'ǽ',
 'ǿ',
 'Ȃ',
 'ȅ',
 'Ȉ',
 'ȏ',
 'Ȑ',
 'Ȕ',
 'ȕ',
 'Ș',
 'Ț',
 'Ȝ',
 'ȡ',
 'ȣ',
 'Ȥ',
 'ȫ',
 'Ȱ',


512

# Part 2

In [16]:
def solution (data, verbose=False):
    
    def calc_orientation (brick):
        if brick[0].x != brick[1].x:
            return "x"
        elif brick[0].y != brick[1].y:
            return "y"
        elif brick[0].z != brick[1].z:
            return "z"
        else:
            return "0"

    dim_max = [0,0,0]
    points = [point for brick in data for point in brick]
    for i in range(3):
        dim_max[i] = max([point[i] for point in points])+1


    structure = np.zeros(dim_max, dtype=str)
    stack = np.zeros(dim_max[:2], dtype=int)

    for idx, brick in enumerate(data):
        if verbose:
            print(brick)
        orientation = calc_orientation(brick)
        if orientation in ("x", "0"):
            y = brick[0].y
            max_z = max(stack[brick[0].x:brick[1].x+1, y])
            stack[brick[0].x:brick[1].x+1, y] =  max_z + 1 
            
        elif orientation == "y":
            x = brick[0].x
            max_z = max(stack[x, brick[0].y:brick[1].y+1])
            stack[x, brick[0].y:brick[1].y+1] =  max_z + 1 

        elif orientation == "z":
            x = brick[0].x
            y = brick[0].y
            max_z = stack[x,y]
            stack[x,y] = max_z + (brick[1].z - brick[0].z + 1)
        
        structure[brick[0].x : brick[1].x+1, brick[0].y : brick[1].y+1, max_z+1 : max_z+1 + (brick[1].z - brick[0].z + 1)  ] = chr(ord("A") +idx)
        # print(structure[0,:,:])
        # print(structure[:,0,:])

    support_map = defaultdict(list)
    supporting_map = defaultdict(list)


    for idx, brick in enumerate(data):
        label = chr(ord("A") +idx)
        locations = np.where(structure == label)
        
        supporting = np.unique(structure[locations[0], locations[1], locations[2]+1]).tolist()
        if "" in supporting:
            supporting.remove("")
        if label in supporting:
            supporting.remove(label)

        supporting_map[label] = supporting

        for sup in supporting:
            if not sup in ("", label):
                support_map[sup] = support_map[sup] + [label]
        if verbose:
            print(label, "supports", supporting)
    
    if verbose:
        display(support_map)


    
    chain_fall = {}
    for idx, brick in enumerate(data):
        print("Calculating brick", idx, "of", len(data))
        label = chr(ord("A") +idx)
        support_map_cpy = deepcopy(support_map)
        fallen_bricks = []
        for key, val in support_map_cpy.items():
            if label in val:
                val.remove(label)
        
        changes = True
        while changes:
            changes = False
            to_fall = []
            for key, val in support_map_cpy.items():
                if not val:
                    to_fall.append(key)
                    fallen_bricks.append(key)
                    changes = True

            support_map_cpy = {key: val for key, val in support_map_cpy.items() if key not in to_fall }
            
            for key, labels in support_map_cpy.items():
                support_map_cpy[key] = [label for label in labels if label not in to_fall]

        chain_fall[label] = fallen_bricks

    return chain_fall
    

sol  = solution(data, verbose=False)
# sol = solution(test_data, verbose=False)

# display(sol)
display(sum([len(val) for val in sol.values()]))
# display(sum(sol))


Calculating brick 0 of 1499
Calculating brick 1 of 1499
Calculating brick 2 of 1499
Calculating brick 3 of 1499
Calculating brick 4 of 1499
Calculating brick 5 of 1499
Calculating brick 6 of 1499
Calculating brick 7 of 1499
Calculating brick 8 of 1499
Calculating brick 9 of 1499
Calculating brick 10 of 1499
Calculating brick 11 of 1499
Calculating brick 12 of 1499
Calculating brick 13 of 1499
Calculating brick 14 of 1499
Calculating brick 15 of 1499
Calculating brick 16 of 1499
Calculating brick 17 of 1499
Calculating brick 18 of 1499
Calculating brick 19 of 1499
Calculating brick 20 of 1499
Calculating brick 21 of 1499
Calculating brick 22 of 1499
Calculating brick 23 of 1499
Calculating brick 24 of 1499
Calculating brick 25 of 1499
Calculating brick 26 of 1499
Calculating brick 27 of 1499
Calculating brick 28 of 1499
Calculating brick 29 of 1499
Calculating brick 30 of 1499
Calculating brick 31 of 1499
Calculating brick 32 of 1499
Calculating brick 33 of 1499
Calculating brick 34 of 

{'A': ['L'],
 'B': ['V'],
 'C': [],
 'D': ['S', 'a', 'n', 'u'],
 'E': [],
 'F': [],
 'G': ['Z',
  ']',
  '`',
  'b',
  'c',
  't',
  'l',
  'm',
  'z',
  'v',
  '\x84',
  '~',
  '\x7f',
  '\x8a',
  '\x8f',
  '\x83',
  '\x87',
  '\x8d',
  '\x92',
  '\x8e',
  '\x97',
  '\x9d',
  '\x94',
  '\x96',
  '\x9e',
  '\x9b',
  '\x9c',
  '\x93',
  '\xa0',
  '¦',
  'ª',
  '¡',
  '§',
  '¨',
  '£',
  '\x9f',
  'Â',
  '¹',
  '¯',
  '´',
  'µ',
  '\xad',
  '®',
  '¸',
  '«',
  'Á',
  '¶',
  '·',
  '²',
  '¾',
  'Ä',
  'º',
  '¿',
  'Ò',
  '½',
  'Ï',
  '»',
  'É',
  'Å',
  'Ç',
  'Í',
  'Ù',
  'È',
  'Ê',
  'Ë',
  'Ì',
  'Î',
  'Ö',
  'Ô',
  'Ø',
  '×',
  'Þ',
  'Õ',
  'Ð',
  'Ó',
  'Ñ',
  'à',
  'â',
  'Ú',
  'Û',
  'Ý',
  'Ü',
  'ß',
  'æ',
  'ã',
  'ä',
  'å',
  'î',
  'í',
  'ì',
  'ë',
  'á',
  'è',
  'ç',
  '÷',
  'ö',
  'ø',
  'ÿ',
  'û',
  'þ',
  'ñ',
  'é',
  'ô',
  'ï',
  'ð',
  'ý',
  'ć',
  'Ĉ',
  'Ć',
  'ā',
  'ê',
  'õ',
  'ó',
  'ù',
  'ú',
  'Ą',
  'ü',
  'ă',
  'ą',
  'Č',
  'ĉ',
  'č

98167